# GPT Annotation (single article per request)

### Import required libraries

In [1]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
from time import sleep
import os

load_dotenv()

True

## GPT Annotation

### Set API key

In [5]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#### Read input and define output location

In [6]:
input_path = ".././../../../data/raw/articles/guardian/2022_articles_guardian_raw.csv" # can be modified
output_path = "2022_articles_guardian_raw_annotated.csv" # can be modified

df = pd.read_csv(input_path, sep=';')


### Run the annotation

In [ ]:
system_message = "You are an excellent annotator. You will be provided with a title in between <titleStart> and <titleEnd> and one or multiple tags in between <tagStart> and <tagEnd>. Both the title and tags belong to a news article. Your task is to classify to which of the following categories the article most likely belongs to. However, you are only allowed to pick categories from the following list: 'Category-1: politics', 'Category-2: business and economy', 'Category-3: environment', 'Category-4: sports', 'Category-5: entertainment and culture', 'Category-6: science and technology' or 'Category-7: health'. Return the number of only one of the provided categories."

for counter, (index, row) in enumerate(df.iterrows(), start=1):
    print(counter)
    title = row['Title']
    tags = row['Tags'][1:-1]
    user_message = f"<titleStart> {title} <titleEnd>; <tagStart> {tags} <tagEnd>"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0,
        max_tokens=256
    )

    category = response.choices[0].message.content
    print(category)
    
    # Following timeout is only needed for the free tier (uncomment if needed)
    # sleep(21)
    df.loc[index, 'Category'] = category

df.to_csv(output_path, sep=';')